<h1 style="text-align:center">Deep Learning   </h1>
<h1 style="text-align:center"> Lab Session 2 - 1.5 Hours </h1>
<h1 style="text-align:center"> Convolutional Neural Network (CNN) for Handwritten Digits Recognition</h1>

<b> Group name:</b> # Elise  Dong, Ariane Horbach,
Deeplearn44
 
 
The aim of this session is to practice with Convolutional Neural Networks. Each group should fill and run appropriate notebook cells. 


Generate your final report (export as HTML) and upload it on the submission website http://bigfoot-m1.eurecom.fr/teachingsub/login (using your deeplearnXX/password). Do not forget to run all your cells before generating your final report and do not forget to include the names of all participants in the group. The lab session should be completed and submitted by May 30th 2018 (23:59:59 CET).

# Introduction

In the previous Lab Session, you built a Multilayer Perceptron for recognizing hand-written digits from the MNIST data-set. The best achieved accuracy on testing data was about 97%. Can you do better than these results using a deep CNN ?
In this Lab Session, you will build, train and optimize in TensorFlow one of the early Convolutional Neural Networks,  **LeNet-5**, to go to more than 99% of accuracy. 






# Load MNIST Data in TensorFlow
Run the cell below to load the MNIST data that comes with TensorFlow. You will use this data in **Section 1** and **Section 2**.

In [1]:
import tensorflow as tf
import numpy as np
import time
from tensorflow.examples.tutorials.mnist import input_data
mnist = input_data.read_data_sets('.../'+"MNIST_data/", one_hot=True)
X_train, y_train           = mnist.train.images, mnist.train.labels
X_validation, y_validation = mnist.validation.images, mnist.validation.labels
X_test, y_test             = mnist.test.images, mnist.test.labels
print("Image Shape: {}".format(X_train[0].shape))
print("Training Set:   {} samples".format(len(X_train)))
print("Validation Set: {} samples".format(len(X_validation)))
print("Test Set:       {} samples".format(len(X_test)))

epsilon = 1e-10 # this is a parameter you will use later

/usr/local/lib/python3.5/dist-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


Extracting .../MNIST_data/train-images-idx3-ubyte.gz
Extracting .../MNIST_data/train-labels-idx1-ubyte.gz
Extracting .../MNIST_data/t10k-images-idx3-ubyte.gz
Extracting .../MNIST_data/t10k-labels-idx1-ubyte.gz
Image Shape: (784,)
Training Set:   55000 samples
Validation Set: 5000 samples
Test Set:       10000 samples


In [6]:
mnist

Datasets(train=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fb2248730f0>, validation=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fb1a1ef09e8>, test=<tensorflow.contrib.learn.python.learn.datasets.mnist.DataSet object at 0x7fb1a1ef05c0>)

In [16]:
y_train[1]

array([0., 0., 0., 1., 0., 0., 0., 0., 0., 0.])

In [14]:
X_train[0]

array([0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.        ,
       0.        , 0.        , 0.        , 0.        , 0.     

# Section 1 : My First Model in TensorFlow

Before starting with CNN, let's train and test in TensorFlow the example
**y=softmax(Wx+b)** seen in the first lab. 

This model reaches an accuracy of about 92 %.
You will also learn how to launch the TensorBoard https://www.tensorflow.org/get_started/summaries_and_tensorboard to visualize the computation graph, statistics and learning curves. 

<b> Part 1 </b> : Read carefully the code in the cell below. Run it to perform training. 

In [4]:
#STEP 1

# Parameters
learning_rate = 0.1
training_epochs = 100
batch_size = 128
display_step = 1
logs_path = 'log_files/'  # useful for tensorboard

# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [None, 784], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

# Set model weights
W = tf.Variable(tf.zeros([784, 10]), name='Weights')
b = tf.Variable(tf.zeros([10]), name='Bias')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = tf.nn.softmax(tf.matmul(x, W) + b) # Softmax
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    # We use tf.clip_by_value to avoid having too low numbers in the log function
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred, epsilon, 1.0)), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()

#STEP 2 

# Launch the graph for training
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size, shuffle=(i==0))
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op], feed_dict={x: batch_xs, y: batch_ys})
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    summary_writer.flush()

    # Test model
    # Calculate accuracy
    print("Accuracy:", acc.eval({x: mnist.test.images, y: mnist.test.labels}))

Epoch:  01   =====> Loss= 0.589759183
Epoch:  02   =====> Loss= 0.382416810
Epoch:  03   =====> Loss= 0.348814638
Epoch:  04   =====> Loss= 0.331653515
Epoch:  05   =====> Loss= 0.320566527
Epoch:  06   =====> Loss= 0.312745708
Epoch:  07   =====> Loss= 0.306496050
Epoch:  08   =====> Loss= 0.301348280
Epoch:  09   =====> Loss= 0.297495954
Epoch:  10   =====> Loss= 0.294215277
Epoch:  11   =====> Loss= 0.291212338
Epoch:  12   =====> Loss= 0.288705647
Epoch:  13   =====> Loss= 0.286202669
Epoch:  14   =====> Loss= 0.284131975
Epoch:  15   =====> Loss= 0.282426295
Epoch:  16   =====> Loss= 0.280914684
Epoch:  17   =====> Loss= 0.279375355
Epoch:  18   =====> Loss= 0.277669499
Epoch:  19   =====> Loss= 0.276561501
Epoch:  20   =====> Loss= 0.275235734
Epoch:  21   =====> Loss= 0.274281487
Epoch:  22   =====> Loss= 0.273452084
Epoch:  23   =====> Loss= 0.272506601
Epoch:  24   =====> Loss= 0.271589055
Epoch:  25   =====> Loss= 0.270429518
Epoch:  26   =====> Loss= 0.269877336
Epoch:  27  

<b> Part 2  </b>: Using Tensorboard, we can  now visualize the created graph, giving you an overview of your architecture and how all of the major components  are connected. You can also see and analyse the learning curves. 

To launch tensorBoard: 
- Open a Terminal and run the command line **"tensorboard --logdir=lab_2/log_files/"**
- Click on "Tensorboard web interface" in Zoe  


Enjoy It !! 


# Section 2 : The 99% MNIST Challenge !

<b> Part 1 </b> : LeNet5 implementation

You are now more familar with **TensorFlow** and **TensorBoard**. In this section, you are to build, train and test the baseline [LeNet-5](http://yann.lecun.com/exdb/lenet/)  model for the MNIST digits recognition problem.  

Then, you will make some optimizations to get more than 99% of accuracy.

For more informations, have a look at this list of results: http://rodrigob.github.io/are_we_there_yet/build/classification_datasets_results.html


<img src="lenet.png",width="800" height="600" align="center">
<center><span>Figure 1: Lenet-5 </span></center>





The LeNet architecture takes a 28x28xC image as input, where C is the number of color channels. Since MNIST images are grayscale, C is 1 in this case.

--------------------------
**Layer 1 - Convolution (5x5):** The output shape should be 28x28x6. **Activation:** ReLU. **MaxPooling:** The output shape should be 14x14x6.

**Layer 2 - Convolution (5x5):** The output shape should be 10x10x16. **Activation:** ReLU. **MaxPooling:** The output shape should be 5x5x16.

**Flatten:** Flatten the output shape of the final pooling layer such that it's 1D instead of 3D.  You may need to use tf.reshape.

**Layer 3 - Fully Connected:** This should have 120 outputs. **Activation:** ReLU.

**Layer 4 - Fully Connected:** This should have 84 outputs. **Activation:** ReLU.

**Layer 5 - Fully Connected:** This should have 10 outputs. **Activation:** softmax.


In [2]:
#Helper functions  for weigths and bias initilization 
def weight_variable(shape):
    initial = tf.truncated_normal(shape, stddev=0.1)
    return tf.Variable(initial)

def bias_variable(shape):
    initial = tf.constant(0.1, shape=shape)
    return tf.Variable(initial)

The function weight_variable is used to initialize the weights for our Convolutional Network. It takes on the argument shape, which defines the dimension (1-D0 for the output tensor. The tf.truncate_normal function takes on the argument shape as well as a standard deviation. The standard deviation is a 0-D Tensor or Python value (dtype), which can be tweaked to achieve better results.

The bias_variable function is used to initialize the bias for our Convolutional Network. The function takes on shape, which takes the dimensions of resulting tensors. 

In [3]:
# https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks/
# https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/

def LeNet5_Model(data,activation_function=tf.nn.relu):

    # layer 1 param
    conv1_weights = weight_variable([5,5,1,6])
    conv1_bias = bias_variable([6])
    
    # layer 2 param
    conv2_weights = weight_variable([5,5,6,16])
    conv2_bias = bias_variable([16])
    
    # layer 3 param
    layer3_weights = weight_variable([400, 120])
    layer3_bias = bias_variable([120])
    
    # layer 4 param
    layer4_weights = weight_variable([120, 84])
    layer4_bias = bias_variable([84])
    
    # layer 5 param
    layer5_weights = weight_variable([84, 10])
    layer5_bias = bias_variable([10])
    
    
    with tf.name_scope('Model'):
        with tf.name_scope('Layer1'):
            conv1 = tf.nn.conv2d(input=data,filter=conv1_weights,strides=[1,1,1,1],padding='SAME')
            print(conv1.shape)
            relu1 = activation_function(conv1 + conv1_bias)
            pool1 = tf.nn.max_pool(relu1,ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='VALID')
            print(pool1.shape)
            
        with tf.name_scope('Layer2'):
            conv2 = tf.nn.conv2d(input=pool1,filter=conv2_weights,strides=[1,1,1,1],padding='VALID')
            print(conv2.shape)
            relu2 = activation_function(conv2 + conv2_bias)
            pool2 = tf.nn.max_pool(relu2,ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='VALID')   
            print(pool2.shape)
        
        with tf.name_scope('Flatten'):
            flat_inputs = tf.contrib.layers.flatten(pool2)
            print(flat_inputs.shape)
           
        with tf.name_scope('Layer3'):
            out3 = activation_function(tf.matmul(flat_inputs, layer3_weights) + layer3_bias)
            
        with tf.name_scope('Layer4'):
            out4 = activation_function(tf.matmul(out3, layer4_weights) + layer4_bias)
            
        with tf.name_scope('Layer5'):
            pred = tf.nn.softmax(tf.matmul(out4, layer5_weights) + layer5_bias) # Softmax
        return pred

<b> Question 2.1.2. </b>  Calculate the number of parameters of this model 

In [5]:
NParameters_LeNet5 = \
    5*5*1*6 + 6+ \
    5*5*6*16 + 16+ \
    400*120 + \
    120 + \
    120*84 + \
    84 + \
    84*10 + \
    10
    
    
print('Number of parameters in LeNet5: %d'%NParameters_LeNet5)

Number of parameters in LeNet5: 61706


<b> Question 2.1.3. </b>  Define your model, its accuracy and the loss function according to the following parameters (you can look at Section 1 to see what is expected):

     Learning rate: 0.001
     Loss Function: Cross-entropy
     Optimizer: tf.train.GradientDescentOptimizer
     Number of epochs: 40
     Batch size: 128

In [6]:
from __future__ import print_function
import tensorflow as tf
from numpy import array
#STEP 1
tf.reset_default_graph()

# Parameters
learning_rate = 0.001
training_epochs = 40
batch_size = 128
display_step = 1
logs_path = 'log_files/'  # useful for tensorboard

# tf Graph Input:  mnist data image of shape 28*28=784
x = tf.placeholder(tf.float32, [batch_size,28, 28,1], name='InputData')
# 0-9 digits recognition,  10 classes
y = tf.placeholder(tf.float32, [batch_size, 10], name='LabelData')

# Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
with tf.name_scope('Model'):
    # Model
    pred = LeNet5_Model(data=x)
with tf.name_scope('Loss'):
    # Minimize error using cross entropy
    cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(pred), reduction_indices=1))
with tf.name_scope('SGD'):
    # Gradient Descent
    optimizer = tf.train.GradientDescentOptimizer(learning_rate).minimize(cost)
with tf.name_scope('Accuracy'):
    # Accuracy
    acc = tf.equal(tf.argmax(pred, 1), tf.argmax(y, 1))
    acc = tf.reduce_mean(tf.cast(acc, tf.float32))

# Initializing the variables
init = tf.global_variables_initializer()
# Create a summary to monitor cost tensor
tf.summary.scalar("Loss", cost)
# Create a summary to monitor accuracy tensor
tf.summary.scalar("Accuracy", acc)
# Merge all summaries into a single op
merged_summary_op = tf.summary.merge_all()


#STEP 2 


# Launch the graph for training
with tf.Session() as sess:
    sess.run(init)
    # op to write logs to Tensorboard
    summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())
    # Training cycle
    for epoch in range(training_epochs):
        avg_cost = 0.
        total_batch = int(mnist.train.num_examples/batch_size)
        # Loop over all batches
        for i in range(total_batch):
            batch_xs, batch_ys = mnist.train.next_batch(batch_size)

            batch_xs = array(batch_xs).reshape(batch_size, 28,28,1)
            #print(batch_xs.shape)
            #print(batch_xs.dtype)
            # Run optimization op (backprop), cost op (to get loss value)
            # and summary nodes
            _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                     feed_dict={x: batch_xs, y: batch_ys})
            
            # Write logs at every iteration
            summary_writer.add_summary(summary, epoch * total_batch + i)
            # Compute average loss
            avg_cost += c / total_batch
        # Display logs per epoch step
        if (epoch+1) % display_step == 0:
            print("Epoch: ", '%02d' % (epoch+1), "  =====> Loss=", "{:.9f}".format(avg_cost))

    print("Optimization Finished!")
    summary_writer.flush()

(128, 28, 28, 6)
(128, 14, 14, 6)
(128, 10, 10, 16)
(128, 5, 5, 16)
(128, 400)
Epoch:  01   =====> Loss= 2.286717658
Epoch:  02   =====> Loss= 2.229259240
Epoch:  03   =====> Loss= 2.137424312
Epoch:  04   =====> Loss= 1.923655283
Epoch:  05   =====> Loss= 1.439618085
Epoch:  06   =====> Loss= 0.926344206
Epoch:  07   =====> Loss= 0.675434008
Epoch:  08   =====> Loss= 0.547201579
Epoch:  09   =====> Loss= 0.471650872
Epoch:  10   =====> Loss= 0.427180266
Epoch:  11   =====> Loss= 0.392251058
Epoch:  12   =====> Loss= 0.367934332
Epoch:  13   =====> Loss= 0.346097705
Epoch:  14   =====> Loss= 0.328136293
Epoch:  15   =====> Loss= 0.313854165
Epoch:  16   =====> Loss= 0.300735826
Epoch:  17   =====> Loss= 0.285276640
Epoch:  18   =====> Loss= 0.275655913
Epoch:  19   =====> Loss= 0.266494973
Epoch:  20   =====> Loss= 0.258191565
Epoch:  21   =====> Loss= 0.248693671
Epoch:  22   =====> Loss= 0.237663607
Epoch:  23   =====> Loss= 0.233520937
Epoch:  24   =====> Loss= 0.225409347
Epoch:  2

<b> Question 2.1.4. </b>  Implement the evaluation function for accuracy computation 

In [4]:
def evaluate(model, y):
    #your implementation goes here
    correct_prediction = tf.equal(tf.argmax(model,1), tf.argmax(y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, tf.float32))
    #print(accuracy.eval(feed_dict={x: mnist.test.images, y_: mnist.test.labels}))

    return accuracy

<b> Question 2.1.5. </b>  Implement training pipeline and run the training data through it to train the model.

- Before each epoch, shuffle the training set. 
- Print the loss per mini batch and the training/validation accuracy per epoch. (Display results every 10 epochs)
- Save the model after training
- Print after training the final testing accuracy 



In [5]:
# Initializing the variables
def train(learning_rate, training_epochs, batch_size, display_step, optimizer_method=tf.train.GradientDescentOptimizer,activation_function=tf.nn.relu):
    tf.reset_default_graph()
    # Initializing the session 
    
    logs_path = 'log_files/'  # useful for tensorboard

    # tf Graph Input:  mnist data image of shape 28*28=784
    x = tf.placeholder(tf.float32, [None,28,28,1], name='InputData')
    # 0-9 digits recognition,  10 classes
    y = tf.placeholder(tf.float32, [None, 10], name='LabelData')

    # Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
    with tf.name_scope('Model'):
        # Model
        pred = LeNet5_Model(data=x,activation_function=activation_function)
    with tf.name_scope('Loss'):
        # Minimize error using cross entropy
        cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred,-1.0,1.0)), reduction_indices=1))
        #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    with tf.name_scope('SGD'):
        # Gradient Descent
        optimizer = optimizer_method(learning_rate).minimize(cost)
    with tf.name_scope('Accuracy'):
        # Accuracy
        acc = evaluate(pred, y)

        
    # Initializing the variables
    init = tf.global_variables_initializer()
    # Create a summary to monitor cost tensor
    tf.summary.scalar("Loss", cost)
    # Create a summary to monitor accuracy tensor
    tf.summary.scalar("Accuracy", acc)
    # Merge all summaries into a single op
    merged_summary_op = tf.summary.merge_all()

    saver = tf.train.Saver()
    startTime = time.time()
    print ("Start Training!")
    #t0 = time()
    X_train,Y_train = mnist.train.images.reshape((-1,28,28,1)), mnist.train.labels
    X_val,Y_val = mnist.validation.images.reshape((-1,28,28,1)), mnist.validation.labels
    
    # Launch the graph for training
    with tf.Session() as sess:
        sess.run(init)
        # op to write logs to Tensorboard
        summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

        # Training cycle
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(mnist.train.num_examples/batch_size)

            # Loop over all batches
            for i in range(total_batch):
                # train_next_batch shuffle the images by default
                batch_xs, batch_ys = mnist.train.next_batch(batch_size)
                batch_xs = batch_xs.reshape((-1,28,28,1))

                # Run optimization op (backprop), cost op (to get loss value)
                # and summary nodes
                _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                         feed_dict={x: batch_xs, 
                                                    y: batch_ys})
                
                # Write logs at every iteration
                summary_writer.add_summary(summary, epoch * total_batch + i)
                # Compute average loss
                avg_cost += c / total_batch
                

            # Display logs per epoch step
            if (epoch+1) % display_step == 0:
                print("Epoch: ", '%02d' % (epoch+1), "=====> Loss=", "{:.9f}".format(avg_cost))
                                                
                acc_train = acc.eval({x: X_train, y: Y_train})
                print("Epoch: ", '%02d' % (epoch+1), "=====> Accuracy Train=", "{:.9f}".format(acc_train))
                
                acc_val = acc.eval({x: X_val, y: Y_val})
                print("Epoch: ", '%02d' % (epoch+1), "=====> Accuracy Validation=", "{:.9f}".format(acc_val))
        print ("Training Finished!")
        #t1 = time()
        # Save the variables to disk.
        #save_path = saver.save(sess, "model.ckpt")
        #print("Model saved in file: %s" % save_path)

        #Your implementation for testing accuracy after training goes here
        
        X_test,Y_test = mnist.test.images.reshape((-1,28,28,1)),mnist.test.labels

        acc_test = acc.eval({x: X_test, y: Y_test})
        print("Accuracy Test=", "{:.9f}".format(acc_test))
        endTime = time.time()
        print("Time elapsed:",endTime - startTime)
        return acc_train,acc_val,acc_test,endTime - startTime
    

In [10]:
#%time 
train(0.001,40,128,10,optimizer_method=tf.train.GradientDescentOptimizer,activation_function=tf.nn.relu)

(?, 28, 28, 6)
(?, 14, 14, 6)
(?, 10, 10, 16)
(?, 5, 5, 16)
(?, 400)
Start Training!
Epoch:  10 =====> Loss= 0.413092241
Epoch:  10 =====> Accuracy Train= 0.889163613
Epoch:  10 =====> Accuracy Validation= 0.893599987
Epoch:  20 =====> Loss= 0.247280239
Epoch:  20 =====> Accuracy Train= 0.928436339
Epoch:  20 =====> Accuracy Validation= 0.934199989
Epoch:  30 =====> Loss= 0.190800316
Epoch:  30 =====> Accuracy Train= 0.942945480
Epoch:  30 =====> Accuracy Validation= 0.950200021
Epoch:  40 =====> Loss= 0.158061047
Epoch:  40 =====> Accuracy Train= 0.953981817
Epoch:  40 =====> Accuracy Validation= 0.958199978
Training Finished!
Accuracy Test= 0.959100008
Time elapsed: 1202.9821767807007


(0.9539818, 0.9582, 0.9591, 1202.9821767807007)

<b> Question 2.1.6 </b> : Use TensorBoard to visualise and save loss and accuracy curves. 
You will save figures in the folder **"lab_2/MNIST_figures"** and display them in your notebook.

Please put your loss and accuracy curves here.
# Gradient Descent
<img src="MNIST_figures/figure1_GD.png">

<b> Part 2 </b> : LeNET 5 Optimization


<b> Question 2.2.1 </b>

- Retrain your network with AdamOptimizer and then fill the table above:


| Optimizer            |  Gradient Descent  |    AdamOptimizer    |
|----------------------|--------------------|---------------------|
| Testing Accuracy     |       0.9591       |       0.9898        |       
| Training Time        |     1202.9821      |      408.4076       |  

- Which optimizer gives the best accuracy on test data?

**Your answer:** 
Comparing the two, testing accuracy is higher with the Adam optimizer. However, note that we cut down number of epochs the Adam optimizer runs with because at 20 epochs, the results were already better than with Gradient Descent, and some time between 20 and 30 epochs, the optimizer seems to crash so that the accuracy drops significantly. We believe this is due to the value of epsilon, changing this should help. All that said, the training time between the two can't be compared apples to apples--although it seems much faster, it also had ran for half the epochs as gradient descent did.

In [6]:
tf.reset_default_graph()

#%time
train(0.001,20,128,10,optimizer_method=tf.train.AdamOptimizer,activation_function=tf.nn.relu)

(?, 28, 28, 6)
(?, 14, 14, 6)
(?, 10, 10, 16)
(?, 5, 5, 16)
(?, 400)
Start Training!
Epoch:  10 =====> Loss= 0.020745562
Epoch:  10 =====> Accuracy Train= 0.993945479
Epoch:  10 =====> Accuracy Validation= 0.988399982
Epoch:  20 =====> Loss= 0.009232017
Epoch:  20 =====> Accuracy Train= 0.996818185
Epoch:  20 =====> Accuracy Validation= 0.988799989
Training Finished!
Accuracy Test= 0.989799976
Time elapsed: 408.4076406955719


(0.9968182, 0.9888, 0.9898, 408.4076406955719)

# Adam Optimizer
<img src="MNIST_figures/figure2_Adam.png">

<b> Question 2.2.2</b> Try to add dropout (keep_prob = 0.75) before the first fully connected layer. You will use tf.nn.dropout for that purpose. What accuracy do you achieve on testing data?

**Accuracy achieved on testing data:** 
0.991999984

In [10]:
# https://adeshpande3.github.io/adeshpande3.github.io/A-Beginner's-Guide-To-Understanding-Convolutional-Neural-Networks/
# https://adeshpande3.github.io/A-Beginner%27s-Guide-To-Understanding-Convolutional-Neural-Networks-Part-2/

def LeNet5_Model_dropout(data,keep_prob,activation_function=tf.nn.relu):

    # layer 1 param
    conv1_weights = weight_variable([5,5,1,6])
    conv1_bias = bias_variable([6])
    
    # layer 2 param
    conv2_weights = weight_variable([5,5,6,16])
    conv2_bias = bias_variable([16])
    
    # layer 3 param
    layer3_weights = weight_variable([400, 120])
    layer3_bias = bias_variable([120])
    
    # layer 4 param
    layer4_weights = weight_variable([120, 84])
    layer4_bias = bias_variable([84]) 
    
    # layer 5 param
    layer5_weights = weight_variable([84, 10])
    layer5_bias = bias_variable([10])
    
    
    with tf.name_scope('Model'):
        with tf.name_scope('Layer1'):
            conv1 = tf.nn.conv2d(input=data,filter=conv1_weights,strides=[1,1,1,1],padding='SAME')
            print(conv1.shape)
            relu1 = activation_function(conv1 + conv1_bias)
            pool1 = tf.nn.max_pool(relu1,ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='VALID')
            print(pool1.shape)
            
        with tf.name_scope('Layer2'):
            conv2 = tf.nn.conv2d(input=pool1,filter=conv2_weights,strides=[1,1,1,1],padding='VALID')
            print(conv2.shape)
            relu2 = activation_function(conv2 + conv2_bias)
            pool2 = tf.nn.max_pool(relu2,ksize=[1, 2, 2, 1],strides=[1, 2, 2, 1],padding='VALID')   
            print(pool2.shape)
        
        with tf.name_scope('Flatten'):
            flat_inputs = tf.contrib.layers.flatten(pool2)
            print(flat_inputs.shape)
           
        with tf.name_scope('Layer3'): # first fully connected layer
            out3 = activation_function(tf.matmul(flat_inputs, layer3_weights) + layer3_bias) # dropout called here
            out3 = tf.nn.dropout(out3, keep_prob)
            
        with tf.name_scope('Layer4'):
            out4 = activation_function(tf.matmul(out3, layer4_weights) + layer4_bias)
            
        with tf.name_scope('Layer5'):
            pred = tf.nn.softmax(tf.matmul(out4, layer5_weights) + layer5_bias) # Softmax
        return pred

In [9]:
# Initializing the variables
def train_dropout(learning_rate, training_epochs, batch_size, display_step, optimizer_method=tf.train.GradientDescentOptimizer,activation_function=tf.nn.relu):
    tf.reset_default_graph()
    # Initializing the session 
    
    logs_path = 'log_files/'  # useful for tensorboard

    # tf Graph Input:  mnist data image of shape 28*28=784
    x = tf.placeholder(tf.float32, [None,28, 28,1], name='InputData')
    # 0-9 digits recognition,  10 classes
    y = tf.placeholder(tf.float32, [None, 10], name='LabelData')
    
    keep_prob = tf.placeholder(tf.float32)

    # Construct model and encapsulating all ops into scopes, making Tensorboard's Graph visualization more convenient
    with tf.name_scope('Model'):
        # Model
        pred = LeNet5_Model_dropout(x,keep_prob,activation_function=activation_function)
    with tf.name_scope('Loss'):
        # Minimize error using cross entropy
        cost = tf.reduce_mean(-tf.reduce_sum(y*tf.log(tf.clip_by_value(pred,-1.0,1.0)), reduction_indices=1))
        #cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits(logits=pred, labels=y))
    with tf.name_scope('SGD'):
        # Gradient Descent
        optimizer = optimizer_method(learning_rate).minimize(cost)
    with tf.name_scope('Accuracy'):
        # Accuracy
        acc = evaluate(pred, y)

        
    # Initializing the variables
    init = tf.global_variables_initializer()
    # Create a summary to monitor cost tensor
    tf.summary.scalar("Loss", cost)
    # Create a summary to monitor accuracy tensor
    tf.summary.scalar("Accuracy", acc)
    # Merge all summaries into a single op
    merged_summary_op = tf.summary.merge_all()

    saver = tf.train.Saver()
    startTime = time.time()
    print ("Start Training!")
    #t0 = time()
    X_train,Y_train = mnist.train.images.reshape((-1,28,28,1)), mnist.train.labels
    X_val,Y_val = mnist.validation.images.reshape((-1,28,28,1)), mnist.validation.labels
    
    # Launch the graph for training
    with tf.Session() as sess:
        sess.run(init)
        # op to write logs to Tensorboard
        summary_writer = tf.summary.FileWriter(logs_path, graph=tf.get_default_graph())

        # Training cycle
        for epoch in range(training_epochs):
            avg_cost = 0.
            total_batch = int(mnist.train.num_examples/batch_size)

            # Loop over all batches
            for i in range(total_batch):
                # train_next_batch shuffle the images by default
                batch_xs, batch_ys = mnist.train.next_batch(batch_size)
                batch_xs = batch_xs.reshape((-1,28,28,1))

                # Run optimization op (backprop), cost op (to get loss value)
                # and summary nodes
                _, c, summary = sess.run([optimizer, cost, merged_summary_op],
                                         feed_dict={x: batch_xs, 
                                                    y: batch_ys,keep_prob:0.75})
                
                # Write logs at every iteration
                summary_writer.add_summary(summary, epoch * total_batch + i)
                # Compute average loss
                avg_cost += c / total_batch
                

            # Display logs per epoch step
            if (epoch+1) % display_step == 0:
                print("Epoch: ", '%02d' % (epoch+1), "=====> Loss=", "{:.9f}".format(avg_cost))
                                                
                acc_train = acc.eval({x: X_train, y: Y_train,keep_prob:1.0})
                print("Epoch: ", '%02d' % (epoch+1), "=====> Accuracy Train=", "{:.9f}".format(acc_train))
                
                acc_val = acc.eval({x: X_val, y: Y_val,keep_prob:1.0})
                print("Epoch: ", '%02d' % (epoch+1), "=====> Accuracy Validation=", "{:.9f}".format(acc_val))
        print ("Training Finished!")
        #t1 = time()
        # Save the variables to disk.
        #save_path = saver.save(sess, "model.ckpt")
        #print("Model saved in file: %s" % save_path)

        #Your implementation for testing accuracy after training goes here
        
        X_test,Y_test = mnist.test.images.reshape((-1,28,28,1)),mnist.test.labels

        acc_test = acc.eval({x: X_test, y: Y_test,keep_prob:1.0})
        print("Accuracy Test=", "{:.9f}".format(acc_test))
        endTime = time.time()
        print("Time elapsed:",endTime - startTime)
        return acc_train,acc_val,acc_test,endTime - startTime

In [11]:
train_dropout(0.001,20,128,10,optimizer_method=tf.train.AdamOptimizer,activation_function=tf.nn.relu)

(?, 28, 28, 6)
(?, 14, 14, 6)
(?, 10, 10, 16)
(?, 5, 5, 16)
(?, 400)
Start Training!
Epoch:  10 =====> Loss= 0.026900856
Epoch:  10 =====> Accuracy Train= 0.995163620
Epoch:  10 =====> Accuracy Validation= 0.988399982
Epoch:  20 =====> Loss= 0.012914957
Epoch:  20 =====> Accuracy Train= 0.999036372
Epoch:  20 =====> Accuracy Validation= 0.991999984
Training Finished!
Accuracy Test= 0.991999984
Time elapsed: 637.7806880474091


(0.9990364, 0.992, 0.992, 637.7806880474091)